In [2]:
import gpxpy
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geopy.distance import geodesic
from pytz import timezone

In [3]:
def parse_gpx_file(filepath):
    gpx_file = open(filepath, 'r')
    gpx = gpxpy.parse(gpx_file)

    data = []
    start_time = '2024-05-27 06:18:49+08:00'
    
    for track in gpx.tracks:
        for segment in track.segments:
            for idx, point in enumerate(segment.points):
                data.append({
                    'time': point.time.astimezone(timezone('Asia/Singapore')) if point.time else start_time,
                    'latitude': point.latitude, 
                    'longitude': point.longitude,
                })

    df = pd.DataFrame(data)
    
    df['prev_latitude'] = df['latitude'].shift(1)
    df['prev_longitude'] = df['longitude'].shift(1)

    df = df.dropna().reset_index(drop=True)

    df['distance'] = df.apply(lambda x: geodesic((x['prev_latitude'], x['prev_longitude']), (x['latitude'], x['longitude'])).meters, axis=1)
    df['accumulated_distance'] = df['distance'].cumsum()
    
    return df

In [4]:
garmin_df = parse_gpx_file('../data/27-05-2023/garmin.gpx')
polar_df = parse_gpx_file('../data/27-05-2023/polar.gpx')
route_df = parse_gpx_file('../data/27-05-2023/route.gpx')

In [5]:
garmin_df.head()

,time,latitude,longitude,prev_latitude,prev_longitude,distance,accumulated_distance
0,2024-05-27 06:18:49+08:00,1.309699,103.896454,1.309726,103.896459,3.125524,3.125524
1,2024-05-27 06:18:56+08:00,1.309665,103.896435,1.309699,103.896454,4.252819,7.378343
2,2024-05-27 06:18:57+08:00,1.309670,103.896451,1.309665,103.896435,1.854305,9.232648
3,2024-05-27 06:19:03+08:00,1.309684,103.896497,1.309670,103.896451,5.289423,14.522071
4,2024-05-27 06:19:06+08:00,1.309688,103.896490,1.309684,103.896497,0.834096,15.356167


In [6]:
polar_df.head()

,time,latitude,longitude,prev_latitude,prev_longitude,distance,accumulated_distance
0,2024-05-27 06:19:27.641000+08:00,1.309362,103.896368,1.309362,103.896368,0.000000,0.000000
1,2024-05-27 06:19:28.642000+08:00,1.309355,103.896353,1.309362,103.896368,1.825025,1.825025
2,2024-05-27 06:19:29.642000+08:00,1.309360,103.896333,1.309355,103.896353,2.293450,4.118475
3,2024-05-27 06:19:30.642000+08:00,1.309362,103.896312,1.309360,103.896333,2.417617,6.536092
4,2024-05-27 06:19:31.641000+08:00,1.309360,103.896298,1.309362,103.896312,1.496057,8.032149


In [7]:
route_df.head()

,time,latitude,longitude,prev_latitude,prev_longitude,distance,accumulated_distance
0,2024-05-27 06:18:49+08:00,1.30945,103.89439,1.30946,103.89443,4.584240,4.584240
1,2024-05-27 06:18:49+08:00,1.30935,103.89386,1.30945,103.89439,60.011815,64.596055
2,2024-05-27 06:18:49+08:00,1.30913,103.89280,1.30935,103.89386,120.446170,185.042225
3,2024-05-27 06:18:49+08:00,1.30892,103.89178,1.30913,103.89280,115.867545,300.909770
4,2024-05-27 06:18:49+08:00,1.30885,103.89146,1.30892,103.89178,36.446473,337.356243


In [9]:
route_df.iloc[-1]['accumulated_distance'] * 2

12865.712318277638